In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import mmcv
import matplotlib
import matplotlib.pyplot as plt
import data
import joblib
import torch
import bbox
from tqdm.auto import tqdm, trange
from bbox import cxcywh2cs
#from mesh import render_vids
from meva.lib.vibe_loss import VIBELoss
from meva.utils.video_config import update_cfg
from meva.utils import image_utils, kp_utils
from meva.utils.renderer import Renderer
from meva.utils.demo_utils import convert_crop_cam_to_orig_img

/home/tormod/Documents/APEC/meva/utils/transformations.py:1927: UserWarning: No module named 'meva.utils._transformations'
  warnings.warn(str(err))


In [3]:
vid_folder = '/media/tormod/Den Lille/Thesis/videos'
anno_folder = '/media/tormod/Den Lille/Thesis/annotations'
#feature_folder = '/media/tormod/Den Lille/Thesis/hmr_features'
feature_folder = '/home/tormod/ucph-erda-home/hmr_features'

cd = data.ClimbingDataset(vid_folder, anno_folder, 'test', feat_folder=feature_folder)
first = cd.get(0)
first.keys()

Reading features for ... IMG_2139 IMG_2140 IMG_2141 IMG_2142 IMG_2320 VID_20210123_091729 VID_20210123_104706 VID_20210123_110129 VID_20210123_111337 VID_20210123_111921 

dict_keys(['raw_imgs', 'norm_imgs', 'features', 'climb_labels', 'kp_2d', 'vid_idx', 'frames', 'bboxes', 'trans', 'inv_trans'])

In [6]:
res = joblib.load('/media/tormod/Den Lille/Thesis/meva_results/meva_output.pkl')

KeyboardInterrupt: 

In [4]:
seqidx = 0
frameid = 6
seqinfo = cd.get(seqidx)
inv_trans = seqinfo['inv_trans'][0]
kp_2d = res['joints2d'][seqidx, frameid]
kp_2d = image_utils.normalize_2d_kp(kp_2d, inv=True)
kp_2d = image_utils.trans_points2d(kp_2d, inv_trans)
plt.imshow(seqinfo['raw_imgs'][frameid])
plt.scatter(kp_2d[:,0], kp_2d[:,1])
plt.show();


NameError: name 'res' is not defined

In [19]:
render_vids(cd, res, '/media/tormod/Den Lille/Thesis/meva_out')

IMG_2139 ...



[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 6750/6750, 36.4 task/s, elapsed: 185s, ETA:     0s
IMG_2140 ...



[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 3600/3600, 22.1 task/s, elapsed: 163s, ETA:     0s
IMG_2141 ...



[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 3420/3420, 21.4 task/s, elapsed: 159s, ETA:     0s
IMG_2142 ...



[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2520/2520, 45.3 task/s, elapsed: 56s, ETA:     0s


In [ ]:
config_file = f"meva/cfg/train_meva_2.yml"
cfg = update_cfg(config_file)
loss = VIBELoss(
    e_loss_weight=cfg.LOSS.KP_2D_W,
    e_3d_loss_weight=cfg.LOSS.KP_3D_W,
    e_pose_loss_weight=cfg.LOSS.POSE_W,
    e_shape_loss_weight=cfg.LOSS.SHAPE_W,
)

In [ ]:
def vibe_loss(seqidx, res, cd):
    pred_seq = res['joints2d'][seqidx]
    pred_seq = image_utils.normalize_2d_kp(pred_seq, inv=True)
    pred_seq = torch.Tensor(pred_seq)

    seq_info = cd.get(seqidx)
    gt_seq = torch.Tensor(seq_info['kp_2d'])
    return loss.keypoint_loss(pred_seq, gt_seq, 1,1)
    
vibe_loss(0, res, cd)

In [ ]:
def trans_points(kp_2d, trans):
    shape = kp_2d.shape
    kp_2d = kp_2d.reshape(-1, shape[-1])
    for n_jt in range(kp_2d.shape[0]):
        kp_2d[n_jt, :2] = image_utils.trans_point2d(kp_2d[n_jt], trans)
    kp_2d = kp_2d.reshape(shape)
    return kp_2d

def convert_kp_seq(kp_seq, src, dst):
    for kp_idx in range(kp_seq.shape[0]):
        kp_seq[kp_idx] = kp_utils.convert_kps(kp_seq[kp_idx], src, dst)

def oks(seqidx, res, cd):
    kpt_oks_sigmas = np.array([.26, .25, .25, .35, .35, .79, .79, .72, .72, .62,.62, 1.07, 1.07, .87, .87, .89, .89])/10.0
    k2 = kpt_oks_sigmas**2
    
    seq_info = cd.get(seqidx)
    inv_trans = seq_info['inv_trans'][0]
    s2 = seq_info['bboxes'][:,-1]**2
    s2 = np.repeat(s2[:,np.newaxis], k2.shape[0], axis=1)
    
    pred_seq = res['joints2d'][seqidx]
    pred_seq = image_utils.normalize_2d_kp(pred_seq, inv=True)
    pred_seq = trans_points(pred_seq, inv_trans)
    pred_seq = np.concatenate((pred_seq, np.ones(pred_seq.shape[:2]+(1,))), axis=2)
    pred_seq = kp_utils.convert_kps(pred_seq, 'spin', 'coco')
    
    gt_seq = seq_info['kp_2d']
    gt_seq = trans_points(gt_seq, inv_trans)
    gt_seq = kp_utils.convert_kps(gt_seq, 'spin', 'coco')
    
    d = np.linalg.norm(pred_seq[:,:,:2] - gt_seq[:,:,:2], axis = 2)
    
    return np.exp(-d/(2*s2*k2))

In [ ]:
oks_res = oks(0, res, cd)
oks_res.mean()

In [ ]:
# TODO compute AP AR https://cocodataset.org/#keypoints-eval